In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
from scipy import ndimage
from matplotlib.patches import Ellipse, Rectangle
import matplotlib.colors as pltc
from dext.postprocessing.detection_visualization import get_text_origin
import colorsys
import random

SMALL_SIZE = 8
MEDIUM_SIZE = 15
BIGGER_SIZE = 20
plt.rc('text', usetex=True)
plt.rc('font', size=MEDIUM_SIZE) # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE, labelsize=MEDIUM_SIZE, linewidth=2) # fontsize of the axes title
plt.rc('xtick', labelsize=MEDIUM_SIZE) # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE) # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE) # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{sfmath} \boldmath \usepackage{bm} \usepackage{amsmath}'

In [8]:
result_dir = '/media/deepan/externaldrive1/project_repos/DEXT_versions/dext/images/results/EFFICIENTDETD0_IntegratedGradients'
saliency_image_paths = 'saliency_image_paths'
ap_curve_linspace = 100

In [9]:
def get_history_file(result_dir, filename):
    if os.path.exists(result_dir):
        file = os.path.join(result_dir, filename)
        if os.path.exists(file):
            return file
        else:
            raise ValueError('File in directory unavailable') 
    else:
        raise ValueError('Result directory unavailable')

def get_image_index_counts(result_dir, filename):
    file = get_history_file(result_dir, filename)
    data = [json.loads(line) for line in open(file, 'r')]
    data = np.array(data)
    all_image_index = list(np.unique(data[:, 0]))
    return len(all_image_index)

def get_data(result_dir, filename):
    file = get_history_file(result_dir, filename)
    data = [json.loads(line) for line in open(file, 'r')]
    return np.array(data)

In [10]:
model_saliency_paths = dict()
model_name_interpretation = (result_dir.split('/')[-1])
model_name = model_name_interpretation.split('_')[0]
interpretation_method = model_name_interpretation.split('_')[1]
print('Model name: %s | Interpretation method: %s' % (model_name, interpretation_method))
print("Number of saliency image paths: ", get_image_index_counts(result_dir, saliency_image_paths))
saliency_image_paths_data = get_data(result_dir, saliency_image_paths)

Model name: EFFICIENTDETD0 | Interpretation method: IntegratedGradients
Number of saliency image paths:  3


/media/deepan/externaldrive1/project_repos/paz_versions/paz_efficientdet/venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/media/deepan/externaldrive1/project_repos/paz_versions/paz_efficientdet/venv/lib/python3.6/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [30]:
# Find unique image ids
column_names = ['image_index', 'object_index', 'box', 'confidence', 'class', 'explaining',
                'boxoffset', 'saliency_path', 'image_path']

count_image_index = np.unique(saliency_image_paths_data[:, column_names.index('image_index')], return_counts=True)
count_image_index

(array(['102731', '109741', '162701'], dtype=object), array([15, 25, 10]))

In [31]:
# Select image index to draw images
data = saliency_image_paths_data[saliency_image_paths_data[:,column_names.index('image_index')] == '102731']

In [ ]:
def get_heatmaps(data):
    class_maps = []
    box_maps1 = []
    box_maps2 = []
    box_maps3 = []
    box_maps4 = []
    class_maps_boxes = []
    box_maps1_boxes = []
    box_maps2_boxes = []
    box_maps3_boxes = []
    box_maps4_boxes = []
    labels = []
    for i in data:
        if i[5] == 'Classification':
            file_path = i[-2]
            file_path = file_path.replace('/scratch/dpadma2s/thesis/results/', 
                                          '/media/deepan/externaldrive1/project_repos/DEXT_versions/other_reports/')
            class_maps.append(np.load(file_path))
            class_maps_boxes.append(i[2])
            labels.append(i[4])
        elif i[5] == 'Boxoffset' and i[6] == 0:
            file_path = i[-2]
            file_path = file_path.replace('/scratch/dpadma2s/thesis/results/', 
                                          '/media/deepan/externaldrive1/project_repos/DEXT_versions/other_reports/')
            box_maps1.append(np.load(file_path))
            box_maps1_boxes.append(i[2])
        elif i[5] == 'Boxoffset' and i[6] == 1:
            file_path = i[-2]
            file_path = file_path.replace('/scratch/dpadma2s/thesis/results/', 
                                          '/media/deepan/externaldrive1/project_repos/DEXT_versions/other_reports/')
            box_maps2.append(np.load(file_path))
            box_maps2_boxes.append(i[2])
        elif i[5] == 'Boxoffset' and i[6] == 2:
            file_path = i[-2]
            file_path = file_path.replace('/scratch/dpadma2s/thesis/results/', 
                                          '/media/deepan/externaldrive1/project_repos/DEXT_versions/other_reports/')
            box_maps3.append(np.load(file_path))
            box_maps3_boxes.append(i[2])
        elif i[5] == 'Boxoffset' and i[6] == 3:
            file_path = i[-2]
            file_path = file_path.replace('/scratch/dpadma2s/thesis/results/', 
                                          '/media/deepan/externaldrive1/project_repos/DEXT_versions/other_reports/')
            box_maps4.append(np.load(file_path))
            box_maps4_boxes.append(i[2])
    return (class_maps, class_maps_boxes, box_maps1, box_maps1_boxes, box_maps2, box_maps2_boxes, 
            box_maps3, box_maps3_boxes, box_maps4, box_maps4_boxes, labels)

In [ ]:
(class_maps, class_maps_boxes, box_maps1, box_maps1_boxes, box_maps2, box_maps2_boxes,
 box_maps3, box_maps3_boxes, box_maps4, box_maps4_boxes, labels) = get_heatmaps(data)